In [ ]:
r"""Protease optimization
 _____  _______  _    _
|  __ \|__   __|| |  | |
| |  | |  | |   | |  | |
| |  | |  | |   | |  | |
| |__| |  | |   | |__| |
|_____/   |_|   |______|

__authors__ = Marco Reverenna & Konstantinos Kalogeropoulus
__copyright__ = Copyright 2025-2026
__research-group__ = DTU Biosustain (Multi-omics Network Analytics) and DTU Bioengineering
__date__ = 15 Aug 2025
__maintainer__ = Marco Reverenna
__email__ = marcor@dtu.dk
__status__ = Dev
"""

In [ ]:
import os
import sys

script_dir = os.getcwd()  # get the current working directory
sys.path.append(os.path.join(script_dir, "../src"))

# my modules
import dbg
import mapping as map
import preprocessing as prep
import compute_statistics as comp_stat

# import libraries
from itertools import combinations
from scipy.stats import gaussian_kde
from pathlib import Path
from upsetplot import UpSet


import json
import math
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
plt.style.use("default")
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = 12

In [ ]:
try:
    # works if you are in a script: __file__ exists
    BASE_DIR = Path(__file__).resolve().parents[2]
except NameError:
    # works if you are in a notebook: __file__ does not exist
    BASE_DIR = Path().resolve()
    # go up until the project folder
    while BASE_DIR.name != "InstaNexus" and BASE_DIR != BASE_DIR.parent:
        BASE_DIR = BASE_DIR.parent

JSON_DIR = BASE_DIR / "json"
INPUT_DIR = BASE_DIR / "inputs"
FASTA_DIR = BASE_DIR / "fasta"
OUTPUTS_DIR = BASE_DIR / "outputs"
FIGURES_DIR = BASE_DIR / "figures"
NOTEBOOKS_DIR = BASE_DIR / "notebooks"
RESULTS_DIR = NOTEBOOKS_DIR / "notebook_results"

In [ ]:
print(f"BASE_DIR: {BASE_DIR}")

In [ ]:
def get_sample_metadata(run, chain="", json_path=JSON_DIR / "sample_metadata.json"):
    with open(json_path, "r") as f:
        all_meta = json.load(f)

    if run not in all_meta:
        raise ValueError(f"Run '{run}' not found in metadata.")

    entries = all_meta[run]

    for entry in entries:
        if entry["chain"] == chain:
            return entry

    raise ValueError(f"No metadata found for run '{run}' with chain '{chain}'.")

In [ ]:
def get_colors_from_run(cat, is_scaffold=False, json_path=JSON_DIR / "colors.json"):
    if not os.path.exists(json_path):
        raise FileNotFoundError(f"Missing color file: {json_path}")

    with open(json_path, "r") as f:
        colors = json.load(f)

    category = cat.split("_")[0].lower()
    key = "scaffold" if is_scaffold else "contig"

    try:
        return colors[category][key]
    except KeyError:
        raise ValueError(f"Color not defined for category '{category}' and key '{key}'")

In [ ]:
def get_combination_name(
    ass_method,
    conf,
    kmer_size,
    size_threshold,
    min_overlap,
    min_identity,
    max_mismatches,
):
    if ass_method == "dbg":
        return f"comb_{ass_method}_c{conf}_ks{kmer_size}_ts{size_threshold}_mo{min_overlap}_mi{min_identity}_mm{max_mismatches}"
    else:
        return f"comb_{ass_method}_c{conf}_ts{size_threshold}_mo{min_overlap}_mi{min_identity}_mm{max_mismatches}"

In [ ]:
def plot_ridgeline_log_kde(
    df,
    protease_column="protease",
    conf_column="conf",
    protease_list=None,
    vertical_gap=5,
    figsize=(12, 12),
    cmap="viridis",
    custom_colors=None,
    title="Confidence score distributions of PSMs per protease in BSA",
    save_svg_path=None,
):

    if protease_list is None:
        protease_list = sorted(df[protease_column].dropna().unique())

    x_vals = np.linspace(0, 1, 500)

    # Choose colors
    if custom_colors is not None:
        colors = [custom_colors.get(p, "gray") for p in protease_list]
    else:
        colors = plt.cm.get_cmap(cmap)(np.linspace(0, 1, len(protease_list)))

    # Compute global minimum for consistent scaling
    all_log_densities = []
    for p in protease_list:
        subset = df[df[protease_column] == p][conf_column].dropna()
        if len(subset) < 2:
            all_log_densities.append(None)
            continue
        kde = gaussian_kde(subset)
        density = kde(x_vals)
        log_density = np.log10(density + 1e-6)
        all_log_densities.append(log_density)

    global_min = np.min([d.min() for d in all_log_densities if d is not None])

    plt.figure(figsize=figsize)
    for i, protease in enumerate(protease_list):
        log_density = all_log_densities[i]
        if log_density is None:
            continue
        log_density_shifted = log_density - global_min
        offset = i * vertical_gap

        plt.plot(x_vals, log_density_shifted + offset, color=colors[i], lw=1.5)
        plt.fill_between(
            x_vals, offset, log_density_shifted + offset, alpha=0.4, color=colors[i]
        )
        plt.text(1.01, offset + 0.5, protease, va="center", fontsize=10)

    plt.xlabel("Confidence", fontsize=12)
    plt.ylabel("")
    plt.title(title, fontsize=14)
    plt.yticks([])
    plt.grid(False)

    ax = plt.gca()
    ax.spines["bottom"].set_color("black")
    ax.spines["left"].set_color("black")
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    plt.tight_layout()

    if save_svg_path:
        plt.savefig(save_svg_path, format="svg")
        # print(f"Plot saved to: {save_svg_path}")

    # plt.show()

In [ ]:
def _json_default(o):

    if isinstance(o, (np.integer,)):
        return int(o)

    if isinstance(o, (np.floating,)):
        if math.isnan(o) or math.isinf(o):
            return None
        return float(o)

    if isinstance(o, (np.bool_,)):
        return bool(o)

    return str(o)


def to_float_or_none(x):
    try:
        xf = float(x)
        if math.isnan(xf) or math.isinf(xf):
            return None
        return xf
    except (TypeError, ValueError):
        return None


def to_int_or_none(x):
    try:
        return int(x)
    except (TypeError, ValueError):
        try:
            xf = float(x)
            if math.isnan(xf) or math.isinf(xf):
                return None
            return int(xf)
        except (TypeError, ValueError):
            return None

In [ ]:
run = "bsa"

meta = get_sample_metadata(run)

protein = meta["protein"]
chain = meta["chain"]
proteases = meta["proteases"]

print(f"Protein: {protein}")
print(f"Chain: {chain}")
print(f"Proteases: {proteases}")

In [ ]:
ass_method = "dbg"
kmer_size = 6
conf = 0.86
size_threshold = 0
min_overlap = 3
min_identity = 0.9
max_mismatches = 12

In [ ]:
params = {
    "ass_method": ass_method,
    "conf": conf,
    "kmer_size": kmer_size,
    "min_overlap": min_overlap,
    "min_identity": min_identity,
    "max_mismatches": max_mismatches,
    "size_threshold": size_threshold,
}

### Data cleaning

In [ ]:
protein_norm = prep.normalize_sequence(protein)

df = pd.read_csv(INPUT_DIR / f"{run}.csv")

df["protease"] = df["experiment_name"].apply(
    lambda name: prep.extract_protease(name, proteases)
)

df = prep.clean_dataframe(df)

df["cleaned_preds"] = df["preds"].apply(prep.remove_modifications)

cleaned_psms = df["cleaned_preds"].tolist()

filtered_psms = prep.filter_contaminants(
    cleaned_psms, run, FASTA_DIR / "contaminants.fasta"
)

df = df[df["cleaned_preds"].isin(filtered_psms)]

df["mapped"] = df["cleaned_preds"].apply(
    lambda x: "True" if x in protein_norm else "False"
)

In [ ]:
with open(JSON_DIR / "protease_colors.json", "r") as f:
    colors = json.load(f)

plot_ridgeline_log_kde(
    df,
    protease_column="protease",
    conf_column="conf",
    protease_list=proteases,
    custom_colors=colors,
    save_svg_path=FIGURES_DIR / "confidence_ridgeline.svg",
)

In [ ]:
df = df[df["conf"] > conf]

df.reset_index(drop=True, inplace=True)

final_psms = df["cleaned_preds"].tolist()

In [ ]:
final_psms[0:5]  # Display first 5 PSMs for verification

In [ ]:
df.head()

## Protease optimization

### Forward/accumulation analysis

In [ ]:
# Order proteases by frequency (most frequent first)
ordered_proteases = df["protease"].value_counts().index.tolist()

build_results = []

We iterate the assembly process by adding one more protease each time and we calculate the coverage.

By considering the best combination obtained from the grid search we set the confidence threshold.

In [ ]:
for i in range(1, len(ordered_proteases) + 1):
    # Select the first i proteases from the ordered list
    selected_proteases = ordered_proteases[:i]
    filtered_df = df[df["protease"].isin(selected_proteases)]

    # Extract sequences from the filtered DataFrame (using column "preds")
    seqs = filtered_df["cleaned_preds"].tolist()

    kmers = dbg.get_kmers(seqs, kmer_size=kmer_size)
    edges = dbg.get_debruijn_edges_from_kmers(kmers)

    assembled_contigs = dbg.assemble_contigs(edges)
    assembled_contigs = sorted(assembled_contigs, key=len, reverse=True)
    assembled_contigs = list(set(assembled_contigs))
    assembled_contigs = [seq for seq in assembled_contigs if len(seq) > size_threshold]
    assembled_contigs = sorted(assembled_contigs, key=len, reverse=True)

    mapped_contigs = map.process_protein_contigs_scaffold(
        assembled_contigs, protein_norm, max_mismatches, min_identity
    )
    df_contigs = map.create_dataframe_from_mapped_sequences(data=mapped_contigs)

    os.makedirs(RESULTS_DIR, exist_ok=True)

    stat_contigs = comp_stat.compute_assembly_statistics(
        df=df_contigs,
        sequence_type="contigs",
        output_folder=RESULTS_DIR,
        reference=protein_norm,
        **params
    )

    coverage_contigs = stat_contigs.get("coverage")

    assembled_scaffolds = dbg.create_scaffolds(assembled_contigs, min_overlap)
    assembled_scaffolds = list(set(assembled_scaffolds))
    assembled_scaffolds = sorted(assembled_scaffolds, key=len, reverse=True)
    assembled_scaffolds = [
        scaffold for scaffold in assembled_scaffolds if len(scaffold) > size_threshold
    ]
    assembled_scaffolds = dbg.merge_sequences(assembled_scaffolds)
    assembled_scaffolds = list(set(assembled_scaffolds))
    assembled_scaffolds = sorted(assembled_scaffolds, key=len, reverse=True)
    assembled_scaffolds = [
        scaffold for scaffold in assembled_scaffolds if len(scaffold) > size_threshold
    ]

    mapped_scaffolds = map.process_protein_contigs_scaffold(
        assembled_contigs=assembled_scaffolds,
        target_protein=protein_norm,
        max_mismatches=max_mismatches,
        min_identity=min_identity,
    )

    df_scaffolds_mapped = map.create_dataframe_from_mapped_sequences(
        data=mapped_scaffolds
    )

    stat_scaffolds = comp_stat.compute_assembly_statistics(
        df=df_scaffolds_mapped,
        sequence_type="scaffolds",
        output_folder=RESULTS_DIR,
        reference=protein_norm,
        **params
    )

    coverage_scaffolds = stat_scaffolds.get("coverage")

    build_results.append(
        {
            "n_proteases_used": i,
            "coverage_contigs": coverage_contigs,
            "coverage_scaffolds": coverage_scaffolds,
        }
    )

df_build = pd.DataFrame(build_results)

In [ ]:
df_build

In [ ]:
ordered_proteases

In [ ]:
with open(JSON_DIR / "protease_colors.json", "r") as f:
    protease_colors = json.load(f)

delta_values = df_build["coverage_scaffolds"].diff()
delta_values.iloc[0] = df_build["coverage_scaffolds"].iloc[
    0
]  # prima proteasi rispetto a 0

delta_scaffolds_df = pd.DataFrame(
    {"protease": ordered_proteases, "delta_scaffolds": delta_values.values}
)

delta_scaffolds_df = delta_scaffolds_df.sort_values("delta_scaffolds", ascending=False)

total_gain = delta_scaffolds_df["delta_scaffolds"].sum()
delta_scaffolds_df["cum_coverage_pct"] = (
    (100 * delta_scaffolds_df["delta_scaffolds"].cumsum() / total_gain)
    if total_gain != 0
    else 0.0
)

bar_colors = [
    protease_colors.get(prot, "#999999") for prot in delta_scaffolds_df["protease"]
]

fig_pareto_scaff = go.Figure()

# Barre Δ coverage con colori per proteasi
fig_pareto_scaff.add_trace(
    go.Bar(
        x=delta_scaffolds_df["protease"],
        y=delta_scaffolds_df["delta_scaffolds"],
        name="Δ coverage (scaffolds)",
        marker_color=bar_colors,
    )
)

fig_pareto_scaff.add_trace(
    go.Scatter(
        x=delta_scaffolds_df["protease"],
        y=delta_scaffolds_df["cum_coverage_pct"],
        name="Cumulative % coverage gain",
        mode="lines+markers",
        yaxis="y2",
        line=dict(color="black", dash="dot"),
    )
)

fig_pareto_scaff.update_xaxes(
    showline=True, linewidth=1, linecolor="black", mirror=False, showgrid=False
)
fig_pareto_scaff.update_yaxes(
    showline=True, linewidth=1, linecolor="black", mirror=False, showgrid=False
)

fig_pareto_scaff.update_layout(
    title="Scaffolds coverage gain by protease (colored by protease)",
    xaxis_title="Proteases",
    yaxis=dict(title="Δ coverage (scaffolds)"),
    yaxis2=dict(
        title="Cumulative % coverage gain",
        overlaying="y",
        side="right",
        range=[0, 110],
        showline=True,
        linecolor="black",
        linewidth=1,
        mirror=True,
    ),
    template="plotly_white",
    font=dict(size=14, family="Arial, sans-serif", color="black"),
    showlegend=False,
)

fig_pareto_scaff.show()
fig_pareto_scaff.write_image(FIGURES_DIR / f"{run}_pareto_scaffolds_{ass_method}.svg")

### Leave-one-out approach

In [ ]:
proteases

In [ ]:
# proteases = final_df['protease'].unique()

build_results = []
for protease in proteases:
    # Exclude the current protease
    filtered_df = df[df["protease"] != protease]

    # Extract sequences from the filtered DataFrame (assuming the column "preds")
    seqs = filtered_df["preds"].tolist()

    kmers = dbg.get_kmers(seqs, kmer_size=kmer_size)
    edges = dbg.get_debruijn_edges_from_kmers(kmers)

    assembled_contigs = dbg.assemble_contigs(edges)
    assembled_contigs = sorted(assembled_contigs, key=len, reverse=True)
    assembled_contigs = list(set(assembled_contigs))
    assembled_contigs = [seq for seq in assembled_contigs if len(seq) > size_threshold]
    assembled_contigs = sorted(assembled_contigs, key=len, reverse=True)

    mapped_contigs = map.process_protein_contigs_scaffold(
        assembled_contigs, protein_norm, max_mismatches, min_identity
    )
    df_contigs = map.create_dataframe_from_mapped_sequences(data=mapped_contigs)

    mapped_contigs = map.process_protein_contigs_scaffold(
        assembled_contigs=assembled_contigs,
        target_protein=protein_norm,
        max_mismatches=max_mismatches,
        min_identity=min_identity,
    )
    df_contigs_mapped = map.create_dataframe_from_mapped_sequences(data=mapped_contigs)

    # same as before, we do not need to save the statistics in a specific folder
    stat_contigs = comp_stat.compute_assembly_statistics(
        df=df_contigs_mapped,
        sequence_type="contigs",
        output_folder=RESULTS_DIR,
        reference=protein_norm,
        **params
    )
    coverage_contigs = stat_contigs.get("coverage")

    assembled_scaffolds = dbg.create_scaffolds(assembled_contigs, min_overlap)
    assembled_scaffolds = list(set(assembled_scaffolds))
    assembled_scaffolds = sorted(assembled_scaffolds, key=len, reverse=True)
    assembled_scaffolds = [
        scaffold for scaffold in assembled_scaffolds if len(scaffold) > size_threshold
    ]
    assembled_scaffolds = dbg.merge_sequences(assembled_scaffolds)
    assembled_scaffolds = list(set(assembled_scaffolds))
    assembled_scaffolds = sorted(assembled_scaffolds, key=len, reverse=True)
    assembled_scaffolds = [
        scaffold for scaffold in assembled_scaffolds if len(scaffold) > size_threshold
    ]

    mapped_scaffolds = map.process_protein_contigs_scaffold(
        assembled_contigs=assembled_scaffolds,
        target_protein=protein_norm,
        max_mismatches=max_mismatches,
        min_identity=min_identity,
    )

    df_scaffolds_mapped = map.create_dataframe_from_mapped_sequences(
        data=mapped_scaffolds
    )

    stat_scaffolds = comp_stat.compute_assembly_statistics(
        df=df_scaffolds_mapped,
        sequence_type="scaffolds",
        output_folder=RESULTS_DIR,
        reference=protein_norm,
        **params
    )

    coverage_scaffolds = stat_scaffolds.get("coverage")

    build_results.append(
        {
            "excluded_protease": protease,
            "coverage_contigs": coverage_contigs,
            "coverage_scaffolds": coverage_scaffolds,
        }
    )

df_build_2 = pd.DataFrame(build_results)

In [ ]:
df_build_2

In [ ]:
import json
from pathlib import Path

# Load color map from JSON (keys must match the names in df_build_2['excluded_protease'])
with open(JSON_DIR / "protease_colors.json", "r") as f:
    protease_colors = json.load(f)

# Sort for plotting
df_plot = df_build_2.sort_values("coverage_scaffolds", ascending=False).copy()

# Ensure every protease has a color (optional fallback)
default_color = "#999999"
for p in df_plot["excluded_protease"].unique():
    protease_colors.setdefault(p, default_color)

fig = px.bar(
    df_plot,
    x="coverage_scaffolds",
    y="excluded_protease",
    orientation="h",
    color="excluded_protease",  # color by protease
    color_discrete_map=protease_colors,  # use your JSON colors
    labels={
        "coverage_scaffolds": "Scaffold coverage",
        "excluded_protease": "Excluded protease",
    },
    title="Coverage after excluding each protease",
)

# Keep the sorted order on the y-axis
fig.update_yaxes(
    categoryorder="array", categoryarray=df_plot["excluded_protease"].tolist()
)

# Axis styling
fig.update_xaxes(
    showline=True,
    linewidth=1,
    linecolor="black",
    mirror=False,
    showgrid=False,
    range=[0, 1],
)
fig.update_yaxes(
    showline=True, linewidth=1, linecolor="black", mirror=False, showgrid=False
)

fig.update_layout(template="plotly_white", legend_title_text="Protease")

fig.show()
fig.write_image(Path(FIGURES_DIR) / f"{run}_coverage_leave_one_out.svg")

# Upset plot

In [ ]:
df.head()

In [ ]:
upset_proteases = ["Legumain", "Chymotrypsin", "Trypsin", "Elastase"]

In [ ]:
all_combinations = []

# Generate all combinations of the proteases
for r in range(1, len(upset_proteases) + 1):
    all_combinations.extend(combinations(upset_proteases, r))

all_combinations = [list(comb) for comb in all_combinations]

print(f"All combinations of proteases: {all_combinations}")

In [ ]:
coverage_results = []
matrix_rows = []

for combo in all_combinations:
    specific_df = df[df["protease"].isin(combo)]
    filtered_seqs = specific_df["cleaned_preds"].tolist()

    kmers = dbg.get_kmers(filtered_seqs, kmer_size=kmer_size)
    edges = dbg.get_debruijn_edges_from_kmers(kmers)
    contigs = dbg.assemble_contigs(edges)
    contigs = sorted(set(contigs), key=len, reverse=True)
    contigs = [seq for seq in contigs if len(seq) > size_threshold]

    mapped_contigs = map.process_protein_contigs_scaffold(
        contigs, protein_norm, max_mismatches, min_identity
    )
    df_contigs_mapped = map.create_dataframe_from_mapped_sequences(data=mapped_contigs)

    reference_length = len(protein_norm)
    if len(df_contigs_mapped):
        df_contigs_mapped["sequence_length"] = (
            df_contigs_mapped["end"] - df_contigs_mapped["start"] + 1
        )
        covered_positions = set()
        for s, e in zip(df_contigs_mapped["start"], df_contigs_mapped["end"]):
            covered_positions.update(range(int(s) - 1, int(e)))
        coverage = (
            (len(covered_positions) / reference_length) if reference_length else 0.0
        )

        avg_len = to_float_or_none(df_contigs_mapped["sequence_length"].mean())
        min_len = to_int_or_none(df_contigs_mapped["sequence_length"].min())
        max_len = to_int_or_none(df_contigs_mapped["sequence_length"].max())
        total_seq = to_int_or_none(len(df_contigs_mapped))
    else:
        coverage, avg_len, min_len, max_len, total_seq = 0.0, None, None, None, 0

    statistics = {}
    statistics.update(params)
    statistics.update(
        {
            "reference_length": reference_length,
            "total_sequences": total_seq,
            "average_length": avg_len,
            "min_length": min_len,
            "max_length": max_len,
            "coverage": to_float_or_none(coverage),
        }
    )

    protease_str = "_".join(p.lower() for p in combo)
    file_name = f"contigs_{protease_str}_stats.json"
    output_path = os.path.join(RESULTS_DIR, file_name)

    with open(output_path, "w") as f:
        json.dump(statistics, f, indent=4)

    row = {
        prot: 1 if prot in combo else 0
        for prot in ["Legumain", "Chymotrypsin", "Elastase", "Trypsin"]
    }
    row["coverage"] = statistics["coverage"]
    matrix_rows.append(row)

presence_absence_df = pd.DataFrame(matrix_rows)

In [ ]:
presence_absence_df

In [ ]:
ind_cols = ["Legumain", "Chymotrypsin", "Elastase", "Trypsin"]

data_boolean = presence_absence_df.copy()
for col in ind_cols:
    data_boolean[col] = data_boolean[col].astype(bool)

indexed = data_boolean.set_index(ind_cols)

plt.figure(figsize=(14, 25))
upset = UpSet(
    indexed,
    intersection_plot_elements=0,
    totals_plot_elements=0,
    subset_size="count",
    show_counts=False,
    show_percentages=False,
    element_size=50,
    orientation="horizontal",
)

# Catplot con colore personalizzato
upset.add_catplot(
    value="coverage",
    kind="bar",
    color=get_colors_from_run(run, is_scaffold=False),
    width=0.5,
)

axes = upset.plot()

all_axes = plt.gcf().get_axes()
cat_ax = all_axes[-1]

cat_ax.grid(False)
cat_ax.spines["top"].set_visible(False)
cat_ax.spines["right"].set_visible(False)
cat_ax.spines["left"].set_visible(True)
cat_ax.spines["bottom"].set_visible(False)
cat_ax.set_ylim(0, 1.0)

for p in cat_ax.patches:
    cat_ax.annotate(
        f"{p.get_height():.2f}",
        (p.get_x() + p.get_width() / 2, p.get_height()),
        ha="center",
        va="bottom",
        fontsize=10,
        color="black",
    )

pos = cat_ax.get_position()
cat_ax.set_position([pos.x0, pos.y0, pos.width, pos.height * 2.5])

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig(
    FIGURES_DIR / f"{run}_upset_plot.svg", format="svg", dpi=600, bbox_inches="tight"
)

plt.show()